In [ ]:
import os
import sys

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from ast import literal_eval
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

sns.set_theme(style="darkgrid")

# See complete data
# pd.set_option('max_columns', None)
# pd.set_option("max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
# Input the boot file to analyse
folder = 'finals_2020_11_2'

path = "/mnt/d/PowerTAC/PowerTAC2021/experiments/logtool_results/finals_2020/" + folder

In [ ]:
color_book = {'VidyutVanika': 'darkorchid','EWIIS3_2021': 'deepskyblue', 'CrocodileAgent2020': 'yellowgreen', 'SPOT': 'tomato', 'defaultbroker': 'deeppink'}

def get_colors(list_of_brokers):
    return [color_book.get(broker) for broker in list_of_brokers]

In [ ]:
#### Needs pre-processing of this file ####
# Remove all data other than brokers' mwh and price
# replace '],' with '];' (basically change the delimiter for columns as well as for data)
# remove starting indentation in data only

broker_mkt_prices = pd.read_csv(path + "/BrokerMktPrices.csv", delimiter=';')

In [ ]:
list_of_brokers = broker_mkt_prices.columns.tolist()

In [ ]:
def convert(string):
    string = string.replace(' ', '')
    string = string.replace('[', '')
    string = string.replace(']', '')
    mwh, price = string.split(",")
    return [float(mwh), float(price)]

def convert_df(df):
    mwh_list = [convert(x)[0] for x in df]
    price_list = [convert(x)[1] for x in df]
    return mwh_list, price_list

In [ ]:
num_rows = 2
num_cols = 1

fig = make_subplots(rows=num_rows, 
                    cols=num_cols,
                    subplot_titles=('Avg Market-Price per Timeslot Comparison', 'Total Quantity Purchased per Timeslot Comparison'),
                    vertical_spacing=0.10)

for broker in list_of_brokers:
    
    df = broker_mkt_prices[broker].to_numpy()
    mwh_list, price_list = convert_df(df)
    
    broker = broker.replace(' ', '')
    fig.append_trace(go.Scatter(x = np.arange(len(price_list))+360,
                                y = price_list, 
                                name = broker,
                                line=dict(color=color_book.get(broker)),
                                legendgroup = 1
                               ), row = 1, col = 1)

    fig.append_trace(go.Scatter(x = np.arange(len(mwh_list)) + 360,
                                y = mwh_list, 
                                name = broker,
                                line=dict(color=color_book.get(broker)),
                                legendgroup = 1,
                                showlegend=False
                               ), row = 2, col = 1)

fig.update_yaxes(title_text="Price ($)", row = 1, col = 1)
fig.update_yaxes(title_text="Quantity (MWh)", row = 2, col = 1)
fig.update_xaxes(title_text="Timeslot", row = 2, col = 1)

fig.update_layout(
    height=750, 
    width=1500, 
    template='ggplot2',
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=14)

fig.write_html(path + "/Results/BrokerMktPrices.html")